# Introduction

In [71]:
# for manipulate data
import numpy as np
import pandas as pd
import matplotlib
import plotly.express as px
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.switch_backend('Agg')
%matplotlib inline
import seaborn as sns
sns.set_style("darkgrid")
import warnings
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.filterwarnings('ignore')

# for training data
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Overview

In [72]:
# Load dataset
path = r'..\Datasets\Tesla Stock Price\Tasla_Stock_Updated_V2.csv'
df = pd.read_csv(path).drop('Unnamed: 0', axis=1)

# Dataset info
print(f"Statistic data: \n{df.info()}")

# Statistic dataset
print(f"Statistic data: \n{df.describe(include='all')}")

# A few rows at dataset
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2274 entries, 0 to 2273
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2274 non-null   object 
 1   Open    2274 non-null   float64
 2   High    2274 non-null   float64
 3   Low     2274 non-null   float64
 4   Close   2274 non-null   float64
 5   Volume  2274 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 106.7+ KB
Statistic data: 
None
Statistic data: 
              Date         Open         High          Low        Close  \
count         2274  2274.000000  2274.000000  2274.000000  2274.000000   
unique        2274          NaN          NaN          NaN          NaN   
top     2015-01-02          NaN          NaN          NaN          NaN   
freq             1          NaN          NaN          NaN          NaN   
mean           NaN   103.493730   105.771617   101.036327   103.461794   
std            NaN   111.136174   113.606095   108.

,Date,Open,High,Low,Close,Volume
0,2015-01-02,14.858000,14.883333,14.217333,14.620667,71466000
1,2015-01-05,14.303333,14.433333,13.810667,14.006000,80527500
2,2015-01-06,14.004000,14.280000,13.614000,14.085333,93928500
3,2015-01-07,14.223333,14.318667,13.985333,14.063333,44526000
4,2015-01-08,14.187333,14.253333,14.000667,14.041333,51637500
5,2015-01-09,13.928000,13.998667,13.664000,13.777333,70024500
6,2015-01-12,13.536667,13.631333,13.283333,13.480667,89254500
7,2015-01-13,13.554667,13.840667,13.394000,13.616667,67159500
8,2015-01-14,12.388667,13.013333,12.333333,12.846000,173278500
9,2015-01-15,12.966000,13.050000,12.666667,12.791333,78247500


# Data cleaning and preprocessing

In [73]:
# Check missing values
print(f'Missing Values :\n{df.isnull().sum()}')

# Convert columns 'date' into datetime types
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Volume'] = df['Volume'].astype(float)

# Info data
print(df.info())
df.head()

Missing Values :
Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2274 entries, 0 to 2273
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2274 non-null   datetime64[ns]
 1   Open    2274 non-null   float64       
 2   High    2274 non-null   float64       
 3   Low     2274 non-null   float64       
 4   Close   2274 non-null   float64       
 5   Volume  2274 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 106.7 KB
None


,Date,Open,High,Low,Close,Volume
0,2015-01-02,14.858000,14.883333,14.217333,14.620667,71466000.0
1,2015-01-05,14.303333,14.433333,13.810667,14.006000,80527500.0
2,2015-01-06,14.004000,14.280000,13.614000,14.085333,93928500.0
3,2015-01-07,14.223333,14.318667,13.985333,14.063333,44526000.0
4,2015-01-08,14.187333,14.253333,14.000667,14.041333,51637500.0


# Feature Engineering

In [74]:
# Add columns 'Momentum', 'Moving Average', 'Volatility', 'Return'
df['Momentum'] = df['Close'] - df['Close'].shift(1)
df['ma_5'] = df['Close'].rolling(window=5).mean()
df['volatility_5'] = df['Close'].rolling(window=5).std()
df['Return'] = df['Close'].pct_change()

# Drop NaN first row
df = df.dropna().reset_index(drop=True)

# Show few rows
df.head()

,Date,Open,High,Low,Close,Volume,Momentum,ma_5,volatility_5,Return
0,2015-01-08,14.187333,14.253333,14.000667,14.041333,51637500.0,-0.021999,14.163333,0.257331,-0.001564
1,2015-01-09,13.928000,13.998667,13.664000,13.777333,70024500.0,-0.264000,13.994666,0.124975,-0.018802
2,2015-01-12,13.536667,13.631333,13.283333,13.480667,89254500.0,-0.296666,13.889600,0.260455,-0.021533
3,2015-01-13,13.554667,13.840667,13.394000,13.616667,67159500.0,0.136000,13.795867,0.256709,0.010088
4,2015-01-14,12.388667,13.013333,12.333333,12.846000,173278500.0,-0.770667,13.552400,0.446634,-0.056597


# Exploratory Data Analysis

In [75]:
# Plotting Stock Price
numeric_cols = ['Open','High','Low','Close']
for i, col in enumerate(numeric_cols, start=1):
    fig = px.line(df, x="Date", y=f"{col}", title=f"{col} Price Chart")
    fig.show()

# Predict Stock Price with LSTM Architecture

In [76]:
# make sure the data matches the date
df = df.sort_values('Date')

# Change target to last columns 
df = df[['Date', 'Open', 'High', 'Low', 'Volume','Momentum','ma_5','volatility_5','Return', 'Close']]

# Splittinf data to dataset train and dataset test
split_date = df['Date'].quantile(0.8)
dataset_train = df[df['Date'] <= split_date]
dataset_test = df[df['Date'] > split_date]

# Choose features and target
X = dataset_train.iloc[:, 1:].values
y = dataset_train.iloc[:, -1:].values

# Scaling Data
mx = MinMaxScaler()
my = MinMaxScaler()
X_scaled = mx.fit_transform(X)
y_scaled = my.fit_transform(y)

# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(120, len(dataset_train)):
    X_train.append(X_scaled[i-120:i, :])
    y_train.append(y_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape x into 3d
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 9))

In [ ]:
# Build neural networks
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(120, 9)))
model.add(Dropout(0.2))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=128, return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(units=1, activation='relu'))

# Choose optimizer and callbacks
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='mean_squared_error')
early_stop = EarlyStopping(monitor='loss',patience=50,restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', patience=10,factor=0.5,verbose=1)

# Train model
model.fit(X_train, y_train, epochs = 100, batch_size = 32,callbacks=[early_stop,reduce_lr])

Epoch 1/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 21s 273ms/step - loss: 0.0087 - learning_rate: 1.0000e-04
Epoch 2/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 269ms/step - loss: 0.0020 - learning_rate: 1.0000e-04
Epoch 3/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 272ms/step - loss: 0.0018 - learning_rate: 1.0000e-04
Epoch 4/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 15s 276ms/step - loss: 0.0017 - learning_rate: 1.0000e-04
Epoch 5/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 272ms/step - loss: 0.0016 - learning_rate: 1.0000e-04
Epoch 6/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 272ms/step - loss: 0.0016 - learning_rate: 1.0000e-04
Epoch 7/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 15s 284ms/step - loss: 0.0016 - learning_rate: 1.0000e-04
Epoch 8/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 271ms/step - loss: 0.0015 - learning_rate: 1.0000e-04
Epoch 9/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 15s 275ms/step - loss: 0.0015 - learning_rate: 1.0000e-04
Epoch 10/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 14s 267ms/step - loss: 0.0014 - learning_rate: 1.0000e-04
Epoch 11/100
53/53 ━━━━━━━━━━

In [ ]:
# Create x test
real_stock_price = dataset_test['Close'].values.reshape(-1,1)
inputs = df[len(df) - len(dataset_test) - 120:].iloc[:, 1:].values
input_scaled = mx.transform(inputs)
X_test = []
for i in range(120, 574):
    X_test.append(input_scaled[i-120:i, :])
X_test = np.array(X_test)

# Predicting wth test set
predicted_stock_price = model.predict(X_test)
predicted_stock_price = my.inverse_transform(predicted_stock_price)

# Plotting to compare 'Actual' vs 'Predited' stock price
fig = go.Figure()

# Line actual price
fig.add_trace(go.Scatter(
    y=real_stock_price.flatten(),
    mode='lines',
    name='Actual Stock Price',
    line=dict(color='#1f77b4', width=2.5)
))

# Line predited price
fig.add_trace(go.Scatter(
    y=predicted_stock_price.flatten(),
    mode='lines',
    name='Predicted Stock Price',
    line=dict(color='#ff4136', width=2.5, dash='dot')
))

# Layout (Dark Theme)
fig.update_layout(
    title=dict(
        text="'Actual' vs 'Predicted' Tesla Stock Price (LSTM Model)",
        x=0.5,
        font=dict(size=22, color='white')
    ),
    xaxis=dict(
        title='Time (Days)',
        color='white',
        showgrid=False,
        zeroline=False
    ),
    yaxis=dict(
        title='Stock Price',
        color='white',
        showgrid=True,
        gridcolor='rgba(255,255,255,0.1)'
    ),
    paper_bgcolor='#111111',   # warna background luar
    plot_bgcolor='#111111',    # warna area plot
    legend=dict(
        x=0.02, y=0.98,
        font=dict(color='white'),
        bgcolor='rgba(0,0,0,0)'
    ),
    hovermode='x unified',
    margin=dict(l=40, r=40, t=60, b=40),
    height=500
)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=30, label="1M", step="day", stepmode="backward"),
            dict(count=90, label="3M", step="day", stepmode="backward"),
            dict(count=180, label="6M", step="day", stepmode="backward"),
            dict(step="all")
        ]),
        bgcolor='#222',
        activecolor='#ff4136',
        font=dict(color='white')
    )
)

fig.show()

In [ ]:
dataset_test.shape